In [1]:
# procedure for calculating hydraulic confinement along a pressure tunnel 
# procedure will be documented as a design report
#  calculation done at stationed points at a selected interval along tunnel alignment
#  calculation determines the minimum dstance from the stationed point to the terrain surface 

# project name: 'Test'

In [2]:
# ToDo
#   put common functions (e.g. setup, data format conversions) into a separate module/API

In [3]:
# import required python libraires
import numpy as np
from numpy import *
import pandas as pd
import geopandas as gpd
import shapely as sp
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
%matplotlib inline
#import random  #for testing
# check pd and gpd version
#print(pd.__version__)
#print(gpd.__version__)

In [4]:
# python setup for qgis processing
import sys, os
from qgis.core import QgsApplication
from PyQt4.QtGui import QApplication
app = QApplication([], True)  #True -> window display
QgsApplication.setPrefixPath("/usr", True)
QgsApplication.initQgis()
sys.path.append('/usr/share/qgis/python/plugins')  #export PYTHONPATH not needed in start script
from processing.core.Processing import Processing
Processing.initialize() 
#from processing.tools import *  #not needed currently
import processing

In [5]:
# import DSS modules
#import geospatial as geo

In [6]:
# set wd for this procedure 
os.chdir("/home/kaelin_joseph/DSS.HydraulicConfinement/")

In [7]:
# define required input files
DTM = "data/swissALTI3D_.tif"  #DEM with surface topography
Alignment = "data/TestAlignment.p0.csv"  #tunnel alignment

In [8]:
# temporary data
Alignment_SHP ='tmp/TestAlignment.shp'  #alignment shp from Alignment
Alignment_grass = 'tmp/TestAlignmentGrass.csv'  #alignment csv fixed for grass
Alignment_line = "tmp/TestAlignmentLine.shp"  #intermediate data
#Alignment_points = "tmp/TestAlignmentPoints.csv"  #alignment containing station points
#Alignment_points = "test/Line_Point.csv"  #KLK ??
Alignment_points_SHP = "tmp/TestAlignmentPoints.shp"  #alignment shp containing station points
alignment_dtm = "tmp/TestAlignmentDTM2.csv"  #alignment including terrain elevations at station points
buffer_dtm = "tmp/buffer_dtm.csv"
Buffer_SHP = "tmp/TestBuffer.shp"  #buffer shp containing ring grid points at a particular station point
alignment_points_final_shp_path = "tmp/TestAlignmentPointsFinal.shp"
buffer_final_shp_path = "tmp/TestBufferFinal.shp"
buffer_final_csv = "tmp/buffer_final.csv"
Test_buffer_csv = "tmp/TestBuffer.csv"  #buffer written to csv
Test_buffer_csv = "tmp/TestBuffer.csv"  #buffer written to csv
buffer_final_csv= "tmp/buffer_final.csv"  

In [9]:
# define output files


In [10]:
# create alignment_df (dataframe) from alignment csv                         #ToDo JK: make into csv -> df function
#   required alignment input data: x, y, z coordinates at alignment definition points
#   check csv data before creating df (no trailing blank lines, no duplicate lines)
alignment_df = pd.read_csv(Alignment)
# delete row if only NA are present in row
alignment_df = alignment_df.dropna(how = "all")
# round alignment_df to three decimals
alignment_df = alignment_df.round(decimals=3)

In [11]:
# create segmented polyline representing alignment                          #ToDo JK: make into stationing function
# from points in alignment_df
#   grass input for function v.in.line 1) nust have x first and y second; 2) no header 
#   required output data: x_align, y_align at each station point
alignment_df_grass = alignment_df.loc[:,["Easting", "Northing"]]  #x first and y second
# write Alignment_grass
alignment_df_grass.to_csv(Alignment_grass, header=False, index=False)  #no header

# points to line, write output to Alignment_line
processing.runalg("grass7:v.in.lines",Alignment_grass,",",False,
                  "2605158.67928,2621471.50192,1265877.44552,1269377.45567",0,Alignment_line)

# line to station points, ouput to Alignment_points_SHP
processing.runalg("grass7:v.to.points",Alignment_line,"100",1,True,
                  "2603510,2624270,1260650,1274890",-1,0.0001,0,Alignment_points_SHP)

{'output': 'tmp/TestAlignmentPoints.shp'}

In [12]:
# create alignment_points_df from Alignment_points_SHP                       #ToDo JK: make into shp -> df function
#   required output data: id_point, x_align, y_align at each station point
fp = Alignment_points_SHP
alignment_points_df = gpd.read_file(fp)
#alignment_points_df = gpd.GeoDataFrame.from_file(Alignment_points_SHP)  #same result_type

# create columns for Easting and Northing, then delete columns cat_ and geometry                          #ToDo KLK
#alignment_points_df.geometry
alignment_points_df["x"] = alignment_points_df.geometry.x
alignment_points_df["y"] = alignment_points_df.geometry.y
# clean up columns
alignment_points_df = alignment_points_df.drop(columns =["cat_", "geometry"])
alignment_points_df.head()

,x,y
0,2.612072e+06,1.269294e+06
1,2.612143e+06,1.269241e+06
2,2.612213e+06,1.269188e+06
3,2.612284e+06,1.269135e+06
4,2.612355e+06,1.269082e+06


In [13]:
# add variable "id" to alignment_points_df
alignment_points_df["id_point"] = alignment_points_df.index

# add variable "distance_station" to alignment_points_df
alignment_points_df["distance_station"] = np.nan
for n in range(0, len(alignment_points_df)-1):
    alignment_points_df.iloc[n, alignment_points_df.columns.get_loc("distance_station")] = (
        ((alignment_points_df.iloc[n+1]["x"] - alignment_points_df.iloc[n]["x"])**2
        +(alignment_points_df.iloc[n+1]["x"] - alignment_points_df.iloc[n]["x"])**2 )**(0.5) )

# add variable "distance_station_sum" to alignment_points_df
alignment_points_df["distance_station_sum"] = np.nan
for n in range(0, len(alignment_points_df)-1):
    id_point_sel2 = alignment_points_df.loc[(alignment_points_df.id_point.isin(range(0,n+1))), "distance_station"] 
    id_point_sel2= id_point_sel2.tolist()
    alignment_points_df.iloc[n, alignment_points_df.columns.get_loc("distance_station_sum")] = sum(id_point_sel2)

alignment_points_df.head()

,x,y,id_point,distance_station,distance_station_sum
0,2.612072e+06,1.269294e+06,0,100.347786,100.347786
1,2.612143e+06,1.269241e+06,1,100.347786,200.695572
2,2.612213e+06,1.269188e+06,2,100.347786,301.043358
3,2.612284e+06,1.269135e+06,3,100.347786,401.391144
4,2.612355e+06,1.269082e+06,4,100.347786,501.738930


In [14]:
# interpolate alignment elevations at all station points and write to alignment_points_df                 #ToDo KLK
#   join alignment_points_df with alignment_df
#   required output data: z_align at each station point                #ToDo JK: make into z interpolation function

In [15]:
# add variable "distance_alignment" to alignment_df
alignment_df["distance_alignment"] = np.nan
for n in range(0, len(alignment_df) -1):
    alignment_df.iloc[n, alignment_df.columns.get_loc("distance_alignment")] = ((alignment_df.iloc[n+1]["Easting"]-alignment_df.iloc[n]["Easting"])**2 +(alignment_df.iloc[n+1]["Northing"]-alignment_df.iloc[n]["Northing"])**2 )**(0.5)
alignment_df.head()  

,Point,Type,Station,Northing,Easting,Elevation,distance_alignment
0,0,NaN,204+874.1,1269294.122,2612071.548,220.962,442.544958
1,221,PT,205+389.861,1269029.594,2612426.331,203.058,3736.888722
2,EC10:3,EC,209+528.128,1265960.779,2614558.633,218.579,NaN


In [16]:
alignment_points_df.head()

,x,y,id_point,distance_station,distance_station_sum
0,2.612072e+06,1.269294e+06,0,100.347786,100.347786
1,2.612143e+06,1.269241e+06,1,100.347786,200.695572
2,2.612213e+06,1.269188e+06,2,100.347786,301.043358
3,2.612284e+06,1.269135e+06,3,100.347786,401.391144
4,2.612355e+06,1.269082e+06,4,100.347786,501.738930


In [17]:
# join alignment_df to alignment_points_df
alignment_points_df = pd.merge(left= alignment_points_df, right = alignment_df, 
                 left_on = ["x","y"], 
                 right_on = ["Easting","Northing"], how = "left")

# clean up alignment_points_df
try:
    alignment_points_df = (
        alignment_points_df.drop(columns =["Point", "Type", "Northing", "Easting"]) )
except:
    pass

alignment_points_df.head()

,x,y,id_point,distance_station,distance_station_sum,Station,Elevation,distance_alignment
0,2.612072e+06,1.269294e+06,0,100.347786,100.347786,204+874.1,220.962,442.544958
1,2.612143e+06,1.269241e+06,1,100.347786,200.695572,NaN,NaN,NaN
2,2.612213e+06,1.269188e+06,2,100.347786,301.043358,NaN,NaN,NaN
3,2.612284e+06,1.269135e+06,3,100.347786,401.391144,NaN,NaN,NaN
4,2.612355e+06,1.269082e+06,4,100.347786,501.738930,NaN,NaN,NaN


In [18]:
# get id of alignment_points_df where points of alignment_points_df == alignment_df
id_point_sel =  (
    alignment_points_df.loc[(alignment_points_df.Elevation.isin(alignment_df["Elevation"])), "id_point"] )
id_point_sel= id_point_sel.tolist()
id_point_sel

[0, 5, 43]

In [19]:
# Fill in "Elevation" and "distance_alignment" for points of alignment_points_df where points of 
# alignment_points_df != alignment_df

for n in range(0, len(id_point_sel)-1): 
   alignment_points_df.loc[(alignment_points_df.id_point.isin(range(id_point_sel[n]+1, id_point_sel[n+1]))), "Elevation"] = alignment_df["Elevation"][n]
alignment_points_df

for n in range(0, len(id_point_sel)-1): 
   alignment_points_df.loc[(alignment_points_df.id_point.isin(range(id_point_sel[n]+1, id_point_sel[n+1]))), "distance_alignment"] = alignment_df["distance_alignment"][n]
alignment_points_df.head()

,x,y,id_point,distance_station,distance_station_sum,Station,Elevation,distance_alignment
0,2.612072e+06,1.269294e+06,0,100.347786,100.347786,204+874.1,220.962,442.544958
1,2.612143e+06,1.269241e+06,1,100.347786,200.695572,NaN,220.962,442.544958
2,2.612213e+06,1.269188e+06,2,100.347786,301.043358,NaN,220.962,442.544958
3,2.612284e+06,1.269135e+06,3,100.347786,401.391144,NaN,220.962,442.544958
4,2.612355e+06,1.269082e+06,4,100.347786,501.738930,NaN,220.962,442.544958


In [20]:
# Add variable "distance_station_sum2" to alignment_points_df

alignment_points_df["distance_station_sum2"] = np.nan

for n in range(1, len(id_point_sel)-1): 
    alignment_points_df.loc[(alignment_points_df.id_point.isin(range(id_point_sel[n], id_point_sel[n+1]))), "distance_station_sum2"] = alignment_points_df["distance_station_sum"] - alignment_df["distance_alignment"][n-1]

for n in range(0, 1): 
    alignment_points_df.loc[(alignment_points_df.id_point.isin(range(id_point_sel[n], id_point_sel[n+1]))), "distance_station_sum2"] =  alignment_points_df["distance_station_sum"]
    
#alignment_points_df.head()
alignment_points_df

,x,y,id_point,distance_station,distance_station_sum,Station,Elevation,distance_alignment,distance_station_sum2
0,2.612072e+06,1.269294e+06,0,100.347786,100.347786,204+874.1,220.962,442.544958,100.347786
1,2.612143e+06,1.269241e+06,1,100.347786,200.695572,NaN,220.962,442.544958,200.695572
2,2.612213e+06,1.269188e+06,2,100.347786,301.043358,NaN,220.962,442.544958,301.043358
3,2.612284e+06,1.269135e+06,3,100.347786,401.391144,NaN,220.962,442.544958,401.391144
4,2.612355e+06,1.269082e+06,4,100.347786,501.738930,NaN,220.962,442.544958,501.738930
5,2.612426e+06,1.269030e+06,5,79.356063,581.094994,205+389.861,203.058,3736.888722,138.550036
6,2.612482e+06,1.268949e+06,6,79.356063,660.451057,NaN,203.058,3736.888722,217.906099
7,2.612539e+06,1.268868e+06,7,79.356063,739.807120,NaN,203.058,3736.888722,297.262162
8,2.612595e+06,1.268787e+06,8,79.356063,819.163184,NaN,203.058,3736.888722,376.618226
9,2.612651e+06,1.268707e+06,9,79.356063,898.519247,NaN,203.058,3736.888722,455.974289


In [21]:
# Add variable "z_alignment" to alignment_points_df

alignment_points_df["z_alignment"] = np.nan

for i in range(0, len(alignment_points_df)): # why len(alignment_d -1? Because the distance cannot be calculated for the last point
    if i in id_point_sel:
        alignment_points_df.iloc[i, alignment_points_df.columns.get_loc("z_alignment")] = alignment_points_df.iloc[i]["Elevation"]

    else:
        
        id_point_sel2 = id_point_sel
        id_point_sel2.append(i)
        id_point_sel2.sort()
        
        m = id_point_sel2.index(i)+1     
        n = id_point_sel2[m]  
        o = id_point_sel2.index(i)-1
        p = id_point_sel2[o]
        
        alignment_points_df.iloc[i, alignment_points_df.columns.get_loc("z_alignment")] = alignment_points_df.iloc[p]["Elevation"]+(alignment_points_df.iloc[n]["Elevation"]-alignment_points_df.iloc[p]["Elevation"])/alignment_points_df.iloc[i]["distance_alignment"]*alignment_points_df.iloc[i-1]["distance_station_sum2"]
    
#         print("i:",i)
#         print(id_point_sel2)
#         print("m:", m)
#         print("n:",n)
#         print("o:", o)
#         print("p:",p)
#         print("Elevation n:",alignment_points_df.iloc[n]["Elevation"])
#         #print("Elevation Test:", alignment_df.loc[(alignment_df["id"] == n), "Elevation"]) # maybe above mentioned code can be replaced through this
#         print("Elevation n:",alignment_points_df.iloc[p]["Elevation"])
#         print("\n")
        
        id_point_sel2.remove(i)

alignment_points_df.head()

,x,y,id_point,distance_station,distance_station_sum,Station,Elevation,distance_alignment,distance_station_sum2,z_alignment
0,2.612072e+06,1.269294e+06,0,100.347786,100.347786,204+874.1,220.962,442.544958,100.347786,220.962000
1,2.612143e+06,1.269241e+06,1,100.347786,200.695572,NaN,220.962,442.544958,200.695572,216.902239
2,2.612213e+06,1.269188e+06,2,100.347786,301.043358,NaN,220.962,442.544958,301.043358,212.842478
3,2.612284e+06,1.269135e+06,3,100.347786,401.391144,NaN,220.962,442.544958,401.391144,208.782717
4,2.612355e+06,1.269082e+06,4,100.347786,501.738930,NaN,220.962,442.544958,501.738930,204.722956


In [22]:
# Add variable "z_dtm_alignment" to alignment_points_df 

# alignment_points_df to alignment_points_final_shp
alignment_points_df_geometry = [sp.geometry.Point(row['x'], row['y']) for key, row in alignment_points_df.iterrows()]
alignment_points_crs = {'init': 'epsg:2056'}  #define crs
alignment_points_final_shp = gpd.GeoDataFrame(alignment_points_df, geometry=alignment_points_df_geometry, crs = alignment_points_crs)
alignment_points_final_shp.to_file(alignment_points_final_shp_path, driver='ESRI Shapefile') 
alignment_points_final_shp.head()

# get DTM value for alignment_points                                      #ToDo JK: make into what.points function
processing.runalg("grass7:r.what.points",DTM,alignment_points_final_shp_path, "NA",",", 500,True,False,False,False,False,"2603510.0,2624270.0,1260650.0,1274890.0",-1,0.0001,alignment_dtm)

# create alignment_df (dataframe) from alignment csv 
alignment_dtm_df = pd.read_csv(alignment_dtm)
alignment_dtm_df.head()

# rename tmp... to DTM
alignment_dtm_df_coleqtmp = [col for col in alignment_dtm_df.columns if 'tmp' in col]
if len(alignment_dtm_df_coleqtmp) != 1:
    print "Extraction of DTM col=tmp did not work properly. Please check"
    exit()
alignment_dtm_df = alignment_dtm_df.rename(
    columns= {alignment_dtm_df_coleqtmp[0]: "z_dtm_alignment"})
alignment_dtm_df.head()

# write alignment_dtm_df["z_dtm_alignment"] to alignment_points_df["z_dtm_alignment"]
alignment_points_df["z_dtm_alignment"] = alignment_dtm_df["z_dtm_alignment"]
alignment_points_df.head()

This can cause unexpected results.


,x,y,id_point,distance_station,distance_station_sum,Station,Elevation,distance_alignment,distance_station_sum2,z_alignment,geometry,z_dtm_alignment
0,2.612072e+06,1.269294e+06,0,100.347786,100.347786,204+874.1,220.962,442.544958,100.347786,220.962000,POINT (2612071.548 1269294.122),253.5240
1,2.612143e+06,1.269241e+06,1,100.347786,200.695572,NaN,220.962,442.544958,200.695572,216.902239,POINT (2612142.5046 1269241.2164),255.2533
2,2.612213e+06,1.269188e+06,2,100.347786,301.043358,NaN,220.962,442.544958,301.043358,212.842478,POINT (2612213.4612 1269188.3108),255.7514
3,2.612284e+06,1.269135e+06,3,100.347786,401.391144,NaN,220.962,442.544958,401.391144,208.782717,POINT (2612284.4178 1269135.4052),255.7720
4,2.612355e+06,1.269082e+06,4,100.347786,501.738930,NaN,220.962,442.544958,501.738930,204.722956,POINT (2612355.3744 1269082.4996),257.9256


In [23]:
# # Add fake variable "z_dtm_alignment" to alignment_points_df

# alignment_points_df["z_dtm_alignment"] = np.random.randint(500, 1000, len(alignment_points_df))
# alignment_points_df.head()

In [24]:
# Add variable "h" to alignment_points_df = overburden depth above station point 

alignment_points_df["h"] = alignment_points_df["z_dtm_alignment"]-alignment_points_df["z_alignment"] 
alignment_points_df.head()
alignment_points_df

,x,y,id_point,distance_station,distance_station_sum,Station,Elevation,distance_alignment,distance_station_sum2,z_alignment,geometry,z_dtm_alignment,h
0,2.612072e+06,1.269294e+06,0,100.347786,100.347786,204+874.1,220.962,442.544958,100.347786,220.962000,POINT (2612071.548 1269294.122),253.5240,32.562000
1,2.612143e+06,1.269241e+06,1,100.347786,200.695572,NaN,220.962,442.544958,200.695572,216.902239,POINT (2612142.5046 1269241.2164),255.2533,38.351061
2,2.612213e+06,1.269188e+06,2,100.347786,301.043358,NaN,220.962,442.544958,301.043358,212.842478,POINT (2612213.4612 1269188.3108),255.7514,42.908922
3,2.612284e+06,1.269135e+06,3,100.347786,401.391144,NaN,220.962,442.544958,401.391144,208.782717,POINT (2612284.4178 1269135.4052),255.7720,46.989283
4,2.612355e+06,1.269082e+06,4,100.347786,501.738930,NaN,220.962,442.544958,501.738930,204.722956,POINT (2612355.3744 1269082.4996),257.9256,53.202644
5,2.612426e+06,1.269030e+06,5,79.356063,581.094994,205+389.861,203.058,3736.888722,138.550036,203.058000,POINT (2612426.331 1269029.594),262.7191,59.661100
6,2.612482e+06,1.268949e+06,6,79.356063,660.451057,NaN,203.058,3736.888722,217.906099,203.633461,POINT (2612482.444210526 1268948.835710526),262.8360,59.202539
7,2.612539e+06,1.268868e+06,7,79.356063,739.807120,NaN,203.058,3736.888722,297.262162,203.963063,POINT (2612538.557421052 1268868.077421053),262.6943,58.731237
8,2.612595e+06,1.268787e+06,8,79.356063,819.163184,NaN,203.058,3736.888722,376.618226,204.292665,POINT (2612594.670631579 1268787.319131579),263.6779,59.385235
9,2.612651e+06,1.268707e+06,9,79.356063,898.519247,NaN,203.058,3736.888722,455.974289,204.622267,POINT (2612650.783842105 1268706.560842105),262.8118,58.189533


In [25]:
# # use make_buffer to get buffer grid points at all station points along alignment                         #ToDo KLK
# #   write buffer points to buffer_df and to alignment_points_df
# #   required output data: id_point, id_buffer, x_buffer, y_buffer, z_dtm_alignment at each buffer point  

# input data  #temporarily static
#point = np.array([2612060, 1268065])  #station point [from alignment_points_df]
#h = 110.0  #overburden depth above station point [calculate from alignment_points_df]

# point = np.array([2612060, 1268065], [2612061, 1268066])  #station point [from alignment_points_df]
# h = [110.0, 111,0]  #overburden depth above station point [calculate from alignment_points_df]


def make_buffer(point, overburden):
    c = 0.5  #buffer radius = c*h
    h = overburden
    res = 5.0  #nominal grid resolution 
#    print("h:"+str(h))
#    print("c:"+str(c))
#    print("res:"+str(res))
    intvls_r = int(h*c / res)  #number of intervals along the buffer radius, close enough
#    print("intvls_r:"+str(intvls_r))
    res_r = h*c / intvls_r  #effective resolution along the radius
    buffer = np.array(point)  #initialize buffer, first item is exactly at station point
    #print("buffer: "+str(buffer))

    # calculate local coordinates for grid along a ring
    for i in range(intvls_r):
#        print("h:"+str(h))
        r = c*h - i*res_r
#        print(r)
        perim = 2 * r * pi 
#        print(perim)
        intvls_c = int(perim/res)  #number of intervals along a ring, close enough
#        print(intvls_c)
        item = np.array([0.0, 0.0])  #initialize
        for j in range(intvls_c):
            item[0] = (sin((2*pi) / intvls_c *(j+1)) *r) + point[0]
            item[1] = (cos((2*pi) / intvls_c *(j+1)) *r) + point[1]
            buffer = np.vstack((buffer, item))  #vstack works with arrays of diff nr of items, append does not        

        #print("r: "+str(r))
        #print("intvls_r: "+str(intvls_r))
        #print("intvls_c: "+str(intvls_c))
        return buffer
        
#buffer = make_buffer(point=point, overburden=h)      
#print(buffer)

In [26]:
# # create shp file with all buffer points                                         #ToDo JK: make df -> shp function
# buffer_points = [sp.geometry.Point(row['Easting'], row['Northing'])
#                  for key, row in buffer_df.iterrows()]
# #len(buffer_points)  #check

# # write buffer to Buffer_SHP as shape file
# buffer_spatial = gpd.GeoDataFrame(buffer_df, geometry=buffer_points, crs = crs)
# buffer_spatial.to_file(Buffer_SHP, driver='ESRI Shapefile') 
# #print(buffer_spatial)  #check
# print(Buffer_SHP)# # use make_buffer to get buffer grid points at all station points along alignment                         #ToDo KLK

alignment_points_df_xy = alignment_points_df.as_matrix(columns=['x','y'])  #station point [from alignment_points_df]
#print(alignment_points_df_xy)
#print(len(alignment_points_df_xy))

# overburden = alignment_points_df_h
alignment_points_df_h = alignment_points_df.as_matrix(columns=['h'])
#print(alignment_points_df_h)
#print(len(alignment_points_df_h))

# buffer_df: preliminary result
buffer_df = pd.DataFrame(columns=["id_point", "x_buffer", "y_buffer"])
#print(buffer_df)

# buffer_final_df: final result
buffer_final_df= pd.DataFrame(columns=["id_point", "x_buffer", "y_buffer"])
#print(buffer_final_df)

# dictionary to which buffer_df_sel are appended
buffer_df_sel = {}

for n in range(0, len(alignment_points_df)): 
    #print(n)
    #print(alignment_points_df_xy[n])
    #print(float(alignment_points_df_h[n]))
    #print(alignment_points_df_h[n])
    buffer_selpoint = make_buffer(point=alignment_points_df_xy[n], overburden=alignment_points_df_h[n])
#    print(buffer_selpoint)
    buffer_df_sel[n] = buffer_df.copy(deep=False)
    buffer_df_sel[n]["id_point"] = [n] * len(buffer_selpoint)  
    buffer_df_sel[n]["z_alignment"] = [alignment_points_df.iloc[n, alignment_points_df.columns.get_loc("z_alignment")]] * len(buffer_selpoint)             
    buffer_df_sel[n]["x_buffer"] = buffer_selpoint[0:,0]
    buffer_df_sel[n]["y_buffer"] = buffer_selpoint[0:,1]
    buffer_final_df = pd.concat([buffer_final_df, buffer_df_sel[n]])
    
# add variable "id_buffer" to buffer_final_df
buffer_final_df = buffer_final_df.reset_index(drop=True)
buffer_final_df["id_buffer"] = buffer_final_df.index    

# save buffer_final_df to csv
buffer_final_df.to_csv(buffer_final_csv, sep=",", na_rep="NaN")

#print(buffer_final_df)


In [27]:
#make_buffer(point=[2612071.548, 1269294.122], overburden=32.562)

In [28]:
# Add variable "z_dtm_buffer" and "dist" to buffer_final_df 

# buffer_final_df to buffer_final_shp
buffer_final_df_geometry = [sp.geometry.Point(row['x_buffer'], row['y_buffer']) for key, row in buffer_final_df.iterrows()]
buffer_final_crs = {'init': 'epsg:2056'}  #define crs
buffer_final_shp = gpd.GeoDataFrame(buffer_final_df, geometry=buffer_final_df_geometry, crs = buffer_final_crs)
buffer_final_shp.to_file(buffer_final_shp_path, driver='ESRI Shapefile') 
buffer_final_shp.head()

# get DTM value for buffer_final                                      #ToDo JK: make into what.points function
processing.runalg("grass7:r.what.points",DTM,buffer_final_shp_path, "NA",",", 500,True,False,False,False,False,"2603510.0,2624270.0,1260650.0,1274890.0",-1,0.0001,buffer_dtm)

# create alignment_df (dataframe) from alignment csv 
buffer_dtm_df = pd.read_csv(buffer_dtm)
buffer_dtm_df.head()

# rename tmp... to DTM
buffer_dtm_df_coleqtmp = [col for col in buffer_dtm_df.columns if 'tmp' in col]
if len(buffer_dtm_df_coleqtmp) != 1:
    print "Extraction of DTM col=tmp did not work properly. Please check"
    exit()
buffer_dtm_df = buffer_dtm_df.rename(
    columns= {buffer_dtm_df_coleqtmp[0]: "z_dtm_buffer"})
buffer_dtm_df.head()

# write buffer_dtm_df["z_dtm"] to buffer_final_df["z_dtm"]
buffer_final_df["z_dtm_buffer"] = buffer_dtm_df["z_dtm_buffer"]
buffer_final_df.head()

# calculate "dist"
buffer_final_df["dist"] = buffer_final_df["z_dtm_buffer"]-buffer_final_df["z_alignment"]

# clean up
buffer_final_df = buffer_final_df.drop(columns =["geometry"])
buffer_final_df.head()

This can cause unexpected results.


,id_point,x_buffer,y_buffer,z_alignment,id_buffer,z_dtm_buffer,dist
0,0,2.612072e+06,1.269294e+06,220.962,0,253.5240,32.5620
1,0,2.612077e+06,1.269310e+06,220.962,1,253.0981,32.1361
2,0,2.612081e+06,1.269307e+06,220.962,2,252.9742,32.0122
3,0,2.612085e+06,1.269304e+06,220.962,3,252.9344,31.9724
4,0,2.612087e+06,1.269299e+06,220.962,4,252.5021,31.5401


In [29]:
# calculate "min_dist"

buffer_final_df["min_dist"] = np.nan

for n in range(0, len(alignment_points_df)):
    buffer_final_df_sel = buffer_final_df.loc[(buffer_final_df["id_point"] == n),]
    dist_idxmin=buffer_final_df_sel['dist'].idxmin()
    buffer_final_df.loc[(buffer_final_df["id_buffer"] == dist_idxmin), "min_dist"] = "MIN"

buffer_final_df.to_csv(buffer_final_csv, header=True, index=False)  #no header
buffer_final_df

buffer_final_df.loc[(buffer_final_df["min_dist"] == "MIN"),]

,id_point,x_buffer,y_buffer,z_alignment,id_buffer,z_dtm_buffer,dist,min_dist
5,0,2.612088e+06,1.269294e+06,220.962000,5,252.2435,31.281500,MIN
43,1,2.612133e+06,1.269258e+06,216.902239,43,252.7796,35.877361,MIN
46,2,2.612213e+06,1.269188e+06,212.842478,46,255.7514,42.908922,MIN
99,3,2.612270e+06,1.269154e+06,208.782717,99,255.7272,46.944483,MIN
122,4,2.612343e+06,1.269059e+06,204.722956,122,255.6772,50.954244,MIN
162,5,2.612400e+06,1.269016e+06,203.058000,162,254.2944,51.236400,MIN
204,6,2.612454e+06,1.268955e+06,203.633461,204,262.5267,58.893239,MIN
228,7,2.612553e+06,1.268843e+06,203.963063,228,262.6526,58.689537,MIN
255,8,2.612617e+06,1.268807e+06,204.292665,255,262.6976,58.404935,MIN
295,9,2.612678e+06,1.268717e+06,204.622267,295,256.4987,51.876433,MIN
